In [1]:
import cv2
import numpy as np

In [3]:
labelsPath = "/home/divyanshu/Desktop/SIH/coco.names"
labels = open(labelsPath).read().strip().split("\n")
weightsPath = "/home/divyanshu/Desktop/SIH/yolov3.weights"
configPath = "/home/divyanshu/Desktop/SIH/yolov3.cfg"
YOLO = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = YOLO.getLayerNames()
ln = [ln[i[0] - 1] for i in YOLO.getUnconnectedOutLayers()]

In [ ]:
vid = cv2.VideoCapture(0)

i = 0
while True:
    i += 1
    fbls = []
    ret, image = vid.read()
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB = True, crop = False)
    YOLO.setInput(blob)
    layerOutputs = YOLO.forward(ln)
    boxes = []
    confidences = []
    classIDs = []
    num = 0
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > 0.6 and labels[classID] == 'car':
                num += 1
                box = detection[0:4]*np.array([w,h,w,h])
                (cX, cY, width, height) = box.astype("int")
                x = int(cX - (width/2))
                y = int(cY - (height/2))
                boxes.append([x,y,int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.5)
    if len(idxs) > 0:
        for i in idxs.flatten():
                if labels[classIDs[i]] == 'car' and confidences[i] > 0.8:
                    (x,y) = (boxes[i][0], boxes[i][1])
                    (w,h) = (boxes[i][2], boxes[i][3])
                    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
                    text = "{}: {:.4f}".format(labels[classIDs[i]], confidences[i])
                    cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
    cv2.imshow("Video", image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
vid.release()
cv2.destroyAllWindows()